# Topic Modeling

## Imports

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit Learn
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import LsiModel
from gensim.models import CoherenceModel

# spaCy
import spacy

# nltk
from nltk.corpus import stopwords
from nltk.tokenize.regexp import RegexpTokenizer


# Silence Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Load Text Libraries


### Load Corpora from spaCy and gensim

In [2]:
nlp = spacy.load('en_core_web_sm')

### Stop Words

In [3]:
def add_stop_words(word_list, list_stop_words):
    for word in word_list:
        list_stop_words.add(word)
        
def remove_stop_words(word_list, list_stop_words):
    for word in word_list:
        list_stop_words.remove(word)

#### Load Stop Words from NLTK and spaCy

In [4]:
stop_words_spacy = set(nlp.Defaults.stop_words)
stop_words_nltk = set(stopwords.words('english'))
full_stop_words = stop_words_spacy.union(stop_words_nltk)

#### Modify Stop Words Based on EDA

In [5]:
words_to_add = ['like', 'know', 'want', 'feel', 'going', 'think', 'reddit', 'imgur', 'pron', 'officially', 'story', 'month', 'week', 'time', 'day', 'year']

add_stop_words(words_to_add, full_stop_words)
remove_stop_words(['not'], full_stop_words)

## Read-In Data

In [6]:
health = pd.read_csv('../data/womens_health_preprocessed.csv', lineterminator='\n')
obsgyn = pd.read_csv('../data/fertility_and_pregnancy_preprocessed.csv')
pospar = pd.read_csv('../data/postpartum_preprocessed.csv')

## Modeling

### Model 1: LDA with Scikit Learn

#### Remove Digits from Text
After initial LDA models, it became clear that, while some numbers may occur frequently in posts, they do not add a lot of value to deriving meaning from the topics. Therefore, digits will be removed from the text.

In [7]:
def remove_digits(df, column1, column2):
    df[column1] = df[column1].str.replace('\d+', '', regex = True)
    df[column2] = df[column2].str.replace('\d+', '', regex = True)
    
# https://stackoverflow.com/questions/47010044/how-to-remove-numeric-characters-present-in-countvectorizer

#### Function to Build LDA Model

In [54]:
def build_lda(df, column, n_topics, stop_words = full_stop_words):
    tvec = TfidfVectorizer(max_df = 0.98, min_df = 0.005, ngram_range = (1,2), stop_words = stop_words)
    dtm = tvec.fit_transform(df[column])
    
    LDA = LatentDirichletAllocation(n_components = n_topics, random_state = 42)
    LDA.fit(dtm)
    
    vocab = tvec.get_feature_names()
    topics = LDA.components_
    
    print(f'The number of topics is {n_topics}.')
    print(f'Log-Likelihood Score: {round(LDA.score(dtm), 3)}')
    print(f'Perplexity Score: {round(LDA.perplexity(dtm), 3)}')
    print('\n')
    
    for index, topic in enumerate(topics):
        print(f'The top 15 words for topic {index}')
        print([vocab[index] for index in topic.argsort()[-15:]])
        print('\n')
    
    return LDA.perplexity(dtm)

In [50]:
def grid_search_sk_lda(df, column, n_topics, step_size, stop_words = full_stop_words):
    for i in range(0, n_topics+1, step_size):
        build_lda(df, column, n_topics, stop_words = full_stop_words)
        print('- - - - - - - - - - - - - - - - - -')

#### Grid Search Num Topics for General Health Data

In [35]:
health_sk_lda = build_lda(health, 'lemma_tokens', 17)

The number of topics is 17.
Log-Likelihood Score: -1142793.108
Perplexity Score: 2347.033


The top 15 words for topic 0
['party', 'instagram', 'man', 'bar', 'view', 'friend', 'girl', 'ex', 'deal', 'gift', 'app', 'favorite', 'boyfriend', 'guy', 'date']


The top 15 words for topic 1
['pelvic', 'fibroid', 'left', 'uterus', 'remove', 'doctor', 'cm', 'surgery', 'ovarian cyst', 'pain', 'ultrasound', 'ovary', 'ovarian', 'delete', 'cyst']


The top 15 words for topic 2
['small', 'notice', 'pain', 'finger', 'bump', 'nipple', 'cup', 'lump', 'labia', 'sex', 'hurt', 'tampon', 'not', 'vagina', 'breast']


The top 15 words for topic 3
['college', 'book', 'not', 'help', 'love', 'live', 'need', 'good', 'home', 'advice', 'new', 'interview', 'tip', 'work', 'job']


The top 15 words for topic 4
['weight gain', 'not', 'gain weight', 'self', 'fat', 'food', 'exercise', 'healthy', 'lose weight', 'diet', 'lose', 'body', 'gain', 'eat', 'weight']


The top 15 words for topic 5
['hormone', 'control pill', 'bc'

#### Grid Search Num Topics for Fertility and Pregnancy

In [52]:
build_lda(obsgyn, 'lemma_tokens', 15)

The number of topics is 15.
Log-Likelihood Score: -3399668.159
Perplexity Score: 1990.693


The top 15 words for topic 0
['need', 'pay', 'pregnancy', 'insurance', 'covid', 'home', 'plan', 'hospital', 'pregnant', 'leave', 'baby', 'birth', 'job', 'not', 'work']


The top 15 words for topic 1
['faint', 'early', 'not', 'line', 'symptom', 'sex', 'late', 'delete', 'pregnancy test', 'negative', 'positive', 'pregnancy', 'pregnant', 'period', 'test']


The top 15 words for topic 2
['experience', 'normal', 'heartbeat', 'tell', 'result', 'today', 'pregnancy', 'not', 'appointment', 'baby', 'test', 'doctor', 'scan', 'blood', 'ultrasound']


The top 15 words for topic 3
['hungry', 'craving', 'lose', 'baby', 'weight gain', 'lbs', 'pound', 'healthy', 'pregnant', 'not', 'pregnancy', 'food', 'gain', 'weight', 'eat']


The top 15 words for topic 4
['feed', 'clothing', 'newborn', 'pack', 'formula', 'cloth', 'bottle', 'hospital', 'milk', 'breastfeed', 'breast', 'bag', 'diaper', 'pump', 'baby']


The top 15

LatentDirichletAllocation(n_components=15, random_state=42)

#### Grid Search Num Topics for Pospartum Data

In [57]:
build_lda(pospar, 'lemma_tokens', n_topics = 30)

The number of topics is 30.
Log-Likelihood Score: -1994441.195
Perplexity Score: 3170.134


The top 15 words for topic 0
['conceive', 'late', 'blood', 'glucose', 'not', 'early', 'negative', 'cycle', 'symptom', 'pregnancy test', 'pregnant', 'positive', 'pregnancy', 'period', 'test']


The top 15 words for topic 1
['pee', 'labor', 'experience', 'blood pressure', 'discharge', 'normal', 'not', 'blood', 'braxton hicks', 'hicks', 'pressure', 'contraction', 'braxton', 'pain', 'cramp']


The top 15 words for topic 2
['mom', 'advance', 'help', 'find', 'read', 'recommendation', 'baby', 'online', 'doula', 'experience', 'thank', 'birth', 'book', 'breastfeed', 'class']


The top 15 words for topic 3
['risk', 'sick', 'vitamin', 'medication', 'covid', 'infection', 'shot', 'cold', 'cough', 'safe', 'not', 'flu', 'doctor', 'pregnant', 'pregnancy']


The top 15 words for topic 4
['need', 'talk', 'try', 'mom', 'thing', 'tell', 'pregnancy', 'husband', 'love', 'life', 'kid', 'child', 'pregnant', 'not', 'bab

3170.1336350744855

### LSA with Gensim

In [58]:
# https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [94]:
lsa_tvec = TfidfVectorizer(max_df = 0.98, min_df = 0.001, ngram_range = (1,2), stop_words = full_stop_words)
X = lsa_tvec.fit_transform(health['lemma_tokens'])
X.shape

(30616, 6724)

In [95]:
svd_model = TruncatedSVD(n_components = 10, random_state = 42)
svd_model.fit(X)

TruncatedSVD(n_components=10, random_state=42)

In [96]:
terms = lsa_tvec.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:15]
    print()
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])



Topic 0: 
period
not
start
pain
pill
sex
control
help
birth
birth control
come
try
doctor
look
normal

Topic 1: 
period
pill
birth control
birth
control
bleed
cramp
pregnancy
pregnant
cycle
iud
blood
test
normal
spot

Topic 2: 
infection
yeast
yeast infection
discharge
uti
vagina
bv
smell
pain
sex
antibiotic
symptom
doctor
vaginal
test

Topic 3: 
hair
shave
yeast
infection
yeast infection
wax
wash
period
grow
dye
birth control
birth
pill
control
shampoo

Topic 4: 
birth control
control
birth
pill
yeast
yeast infection
infection
control pill
friend
pack
effect
work
good
new
start birth

Topic 5: 
pain
cyst
iud
birth
birth control
control
doctor
breast
ovarian
hurt
ovarian cyst
ovary
painful
ultrasound
experience

Topic 6: 
wear
bra
dress
look
breast
size
clothe
smell
buy
boob
makeup
jean
fit
skin
cup

Topic 7: 
iud
period
friend
pain
yeast
yeast infection
infection
cramp
cyst
work
copper
bad
heavy
copper iud
experience

Topic 8: 
iud
sex
bleed
smell
copper
copper iud
mirena
vagina
disc

In [92]:
pospar['subreddit'].value_counts()

BabyBumps               33280
breastfeeding           15593
postpartumdepression      221
Name: subreddit, dtype: int64

In [93]:
len(pospar) * 0.005

245.47